In [1]:
import pandas as pd
import numpy as np
import json

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

In [3]:
leads = pd.read_csv("data/leads_smartmarket.csv", parse_dates=["date"])
leads.head()

,lead_id,date,channel,device
0,10000,2025-09-23,Facebook Ads,Mobile
1,10001,2025-09-04,Instagram Ads,Mobile
2,10002,2025-09-18,LinkedIn,Desktop
3,10003,2025-09-05,LinkedIn,Mobile
4,10004,2025-09-16,Emailing,Desktop


In [4]:
with open("data/campaign_smartmarket.json", "r", encoding="utf-8") as f:
    campaign_data = json.load(f)

campaigns = pd.DataFrame(campaign_data)
campaigns.head()

,campaign_id,channel,cost,impressions,clicks,conversions
0,CAMP01,Emailing,1612,47883,1470,82
1,CAMP02,Facebook Ads,3240,114188,2315,140
2,CAMP03,LinkedIn,6704,107834,2224,174
3,CAMP04,Instagram Ads,3676,149599,5023,352
4,CAMP05,Emailing,1628,95923,2269,146


In [5]:
crm = pd.read_excel("data/crm_smartmarket.xlsx")
crm.head()

,lead_id,company_size,sector,region,status
0,10000,100-500,Education,PAC,MQL
1,10001,1-10,Finance,Hauts-de-France,MQL
2,10002,500-1000,Retail,Pays de la Loire,MQL
3,10003,1-10,Industry,PAC,SQL
4,10004,1-10,Finance,IdF,MQL
